In [3]:
# tell interpreter where to look
from datastructures.Rationals import Rationals as rat
from datastructures.Edgebst import *
from datastructures.DCEL import *
from visualize.VisualizePolygons import *
from algorithms.MakeMonotone import *
from algorithms.triangulate import *

f = open("instances/fpg-poly_0000004900_h2.instance.json")
polygon = json.load(f)
polygon_DCEL = DCEL(polygon["outer_boundary"], polygon["holes"])
make_monotone(polygon_DCEL)
triangulate_monotone_polygon(polygon_DCEL)
drawPolygon(polygon, solution=polygon_DCEL.format_solution(), resolution=10)

Building DCEL...
Finished building DCEL.
Subdividing the polygon into y-monotone pieces...
Finished subdividing the polygon into y-monotone pieces.
Recomputing faces...
Finished recomputing faces.
Triangulating y-monotone pieces...
Recomputing faces...
Finished recomputing faces.
Finished triangulating y-monotone pieces.


OverflowError: Python int too large to convert to C long